# Notebook: Prompt Generation and Configuration

This notebook validates prompt creation, context provider integrations, extension mechanisms, and logging behavior. It follows the structure of the linting build-up notebook to ensure consistent testing.

In [1]:
import sys
from importlib import import_module
import logging
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent.parent
sys.path.insert(0, str(PROJECT_ROOT))

from app.factories.prompt_manager import PromptGeneratorFactory
from app.factories.context_provider import ContextProviderFactory
from app.factories.logging_provider import LoggingProvider
from app.factories.tool_provider import ToolProviderFactory

import_module("app.extensions.prompt_generators")
import_module("app.extensions.context_providers")
import_module("app.extensions.tool_providers")

logging.basicConfig(level=logging.DEBUG)

### Basic System Prompt

A minimal system prompt is created using the `BasicPromptGenerator`. The prompt combines system and agent templates with optional context.

In [2]:
provider = ContextProviderFactory.create("dummy")
generator = PromptGeneratorFactory.create("basic", context_provider=provider)

system_config = {"template": "System instructions"}
agent_config = {"template": "Agent instructions"}

experiment_id = "exp001"  # Set this properly
round_number = 1

try:
    prompt = generator.generate_prompt(
        agent_config=agent_config,
        system_config=system_config,
        experiment_id=experiment_id,
        round=round_number,
    )
    print(prompt)
except Exception as e:
    print("Prompt generation failed:", e)

DEBUG:BasicPromptGenerator:Prompt generation start
DEBUG:DummyContextProvider:Context retrieval start
INFO:DummyContextProvider:Dummy context provided
DEBUG:DummyContextProvider:Context retrieval end
DEBUG:BasicPromptGenerator:Raw context: {}
INFO:BasicPromptGenerator:Prompt generated
DEBUG:BasicPromptGenerator:Prompt content: Agent instructions

System instructions


DEBUG:BasicPromptGenerator:Prompt generation logged


Agent instructions

System instructions




### Multiple Agent-Level Prompts

Prompts for generation, evaluation, and mediation roles are produced to ensure the generator handles varied agent instructions.

In [4]:
agent_configs = {
    "generation": {"template": "Generation prompt"},
    "evaluation": {"template": "Evaluation prompt"},
    "mediation": {"template": "Mediation prompt"},
}

experiment_id = "exp001"  # Your experiment ID
round_number = 1  # Your experiment round

for name, cfg in agent_configs.items():
    try:
        p = generator.generate_prompt(cfg, system_config, experiment_id, round_number)
        print(f"{name}: {p}")
    except Exception as e:
        print(f"{name} failed:", e)

DEBUG:BasicPromptGenerator:Prompt generation start
DEBUG:DummyContextProvider:Context retrieval start
INFO:DummyContextProvider:Dummy context provided
DEBUG:DummyContextProvider:Context retrieval end
DEBUG:BasicPromptGenerator:Raw context: {}
INFO:BasicPromptGenerator:Prompt generated
DEBUG:BasicPromptGenerator:Prompt content: Generation prompt

System instructions


DEBUG:BasicPromptGenerator:Prompt generation logged
DEBUG:BasicPromptGenerator:Prompt generation start
DEBUG:DummyContextProvider:Context retrieval start
INFO:DummyContextProvider:Dummy context provided
DEBUG:DummyContextProvider:Context retrieval end
DEBUG:BasicPromptGenerator:Raw context: {}
INFO:BasicPromptGenerator:Prompt generated
DEBUG:BasicPromptGenerator:Prompt content: Evaluation prompt

System instructions


DEBUG:BasicPromptGenerator:Prompt generation logged
DEBUG:BasicPromptGenerator:Prompt generation start
DEBUG:DummyContextProvider:Context retrieval start
INFO:DummyContextProvider:Dummy context provided
DEBUG

generation: Generation prompt

System instructions


evaluation: Evaluation prompt

System instructions


mediation: Mediation prompt

System instructions




In [6]:
import sqlite3
from pathlib import Path

db_path = Path.cwd().parent.parent / "experiments" / "codecritic.sqlite3"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("SELECT * FROM prompt_generation_log")
prompt_logs = cursor.fetchall()

print("Final Prompt Generation logs in database:", prompt_logs)
conn.close()

Final Prompt Generation logs in database: [('exp001', 1, 'BasicPromptGenerator', 'DummyContextProvider', '{"template": "Agent instructions"}', '{"template": "System instructions"}', 'Agent instructions\n\nSystem instructions\n\n', 1, None, '2025-05-19T03:10:28.427107+00:00')]


### Context Providers

Different context providers supply additional code data. This cell registers available providers and renders prompts with each.

In [7]:
dummy = ContextProviderFactory.create("dummy")
try:
    symbol = ContextProviderFactory.create(
        "symbol_graph", target=Path("sample_module.py")
    )
except Exception as e:
    print("SymbolGraphProvider creation failed:", e)
    symbol = None

providers = [p for p in (dummy, symbol) if p is not None]

experiment_id = "exp001"
round_number = 1

for p in providers:
    try:
        _ = p.get_context()
        result = PromptGeneratorFactory.create(
            "basic", context_provider=p
        ).generate_prompt(agent_config, system_config, experiment_id, round_number)
        print(f"{p.__class__.__name__}: {result}")
    except Exception as e:
        print(p.__class__.__name__, "error:", e)

DEBUG:DummyContextProvider:Context retrieval start
INFO:DummyContextProvider:Dummy context provided
DEBUG:DummyContextProvider:Context retrieval end
DEBUG:BasicPromptGenerator:Prompt generation start
DEBUG:DummyContextProvider:Context retrieval start
INFO:DummyContextProvider:Dummy context provided
DEBUG:DummyContextProvider:Context retrieval end
DEBUG:BasicPromptGenerator:Raw context: {}
INFO:BasicPromptGenerator:Prompt generated
DEBUG:BasicPromptGenerator:Prompt content: Agent instructions

System instructions


DEBUG:BasicPromptGenerator:Prompt generation logged
DEBUG:SymbolGraphProvider:Context retrieval start
ERROR:SymbolGraphProvider:Target not found: sample_module.py
DEBUG:SymbolGraphProvider:Context retrieval end
DEBUG:BasicPromptGenerator:Prompt generation start
DEBUG:SymbolGraphProvider:Context retrieval start
ERROR:SymbolGraphProvider:Target not found: sample_module.py
DEBUG:SymbolGraphProvider:Context retrieval end
DEBUG:BasicPromptGenerator:Raw context: {}
INFO:BasicPrompt

DummyContextProvider: Agent instructions

System instructions


SymbolGraphProvider: Agent instructions

System instructions




### Tool Providers

Tool providers execute external utilities used in the prompting workflow. Errors for invalid providers are explicitly shown.

In [8]:
try:
    black = ToolProviderFactory.create("black")
    tmp = Path("tmp.py")
    tmp.write_text("x=1")
    result = black.run(str(tmp))
    print("Black return:", result.returncode)
except Exception as e:
    print("Black failed:", e)
finally:
    tmp.unlink(missing_ok=True)

try:
    ToolProviderFactory.create("nonexistent")
except Exception as e:
    print("Expected tool error:", e)

DEBUG:BlackToolProvider:Tool run start
DEBUG:BlackToolProvider:Tool run end


Black return: 0
Expected tool error: 'Tool provider nonexistent not registered'


### Logging Verification

Logging operations should succeed under normal conditions and raise errors on misuse.

In [10]:
logger = LoggingProvider(":memory:")
from app.enums.logging_enums import LogType
from app.utilities.metadata.logging.log_schemas import StateLog
from datetime import datetime, timezone

try:
    state_log = StateLog(
        experiment_id="exp",
        system="test_system",  # <-- explicitly required
        round=0,
        state="test",
        action="test_action",  # <-- explicitly required
        timestamp=datetime.now(timezone.utc),
    )
    logger.log_state(state_log)
    print("State logged successfully")
except Exception as e:
    print("Logging failed:", e)

try:
    logger.write(LogType.STATE, object())
except Exception as e:
    print("Expected logging error:", e)

State logged successfully
Expected logging error: Expected StateLog, got <class 'object'>
